In [38]:
import numpy as np 
import pandas as pd 
from ast import literal_eval
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
df = pd.read_csv('./Data generated.csv')

In [40]:
df.head()

,userid,age,gender,domisili,pekerjaan,instansi,hobi
0,61377,18,female,Kota Magelang,Content writer,Universitas Gunadarma,Programming
1,16429,25,male,Kabupaten Bangkalan,Data analyst,Universitas Andalas (UNAND),Menonton
2,27812,54,male,Kota Bogor,Blogger,Universitas Negeri Malang (UM),Menonton
3,30786,39,female,Kabupaten Situbondo,Web developer,Universitas Udayana (UNUD),Mendengarkan musik
4,76660,45,male,Kabupaten Tulungagung,Data analyst,SMA A,Olahraga


In [41]:
# Ubah tipe data userId sebagai Integer
df['userid'] = df['userid'].astype('int')

In [42]:
# Tentukan kriteria rekomendasi pertemanan
df['overall'] = df['domisili'] + ' ' + df['pekerjaan'] + ' ' +df['instansi'] + ' ' + df['hobi']

In [43]:
# Ubah dokumen text ke matriks berdasarkan fitur/pembobotan CountVectorizer
count = CountVectorizer(analyzer = 'word', ngram_range = (1, 2), min_df = 0, 
                        # stop_words = 'english'
                        )
count_matrix = count.fit_transform(df['overall'])

In [44]:
# # Convert dokumen ke matriks berdasarkan fitur/pembobotan TF-IDF
# tf = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df = 0, 
# #                      stop_words='english'
#                     )
# tfidf_matrix = tf.fit_transform(df['overall'])

In [45]:
df = df.reset_index(drop=True)
titles = df[['userid', 'domisili', 'pekerjaan', 'instansi', 'hobi']]  # Tentukan output yang diinginkan dlm rekomendasi pertemanan
indices = pd.Series(df.index, index = df['userid'])  # Buat dataframe baru dengan index userId

In [74]:
# Buat fungsi untuk membuat 10 rekomendasi pertemanan paling mirip/similar berdasarkan cosine similarity score
def get_recommendations(title):
    idx = indices[title]  
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # 10 rekomendasi pertemanan paling mirip/similar
    friend_indices = [i[0] for i in sim_scores]
    similarity_scores = [i[1] for i in sim_scores]  # Dapat similarity scores
    recommended_friends = titles.iloc[friend_indices]  # Dapat teman yang direkomendasikan
    recommended_friends['Similarity Score'] = similarity_scores  # Tambah kolom Similarity Score
    return recommended_friends 

In [75]:
# Saatnya kita tessss
desired_user_id = 61377
df[df['userid'] == desired_user_id]

,userid,age,gender,domisili,pekerjaan,instansi,hobi,overall
0,61377,18,female,Kota Magelang,Content writer,Universitas Gunadarma,Programming,Kota Magelang Content writer Universitas Gunad...


In [76]:
get_recommendations(desired_user_id)

,userid,domisili,pekerjaan,instansi,hobi,Similarity Score
875,30920,Kabupaten Demak,Content writer,Universitas Gunadarma,Programming,0.692308
29,37574,Kota Surakarta,Content writer,Universitas Gunadarma,Menonton,0.615385
503,69712,Kabupaten Pamekasan,Content writer,Universitas Gunadarma,Makan,0.538462
717,70204,Kabupaten Kendal,Content writer,Universitas Gunadarma,Makan,0.538462
319,68263,Kabupaten Nganjuk,Content writer,Universitas Gunadarma,Bermain game,0.501280
465,34405,Kabupaten Trenggalek,Content writer,Universitas Gunadarma,Bermain musik,0.501280
923,50456,Kabupaten Trenggalek,Content writer,Universitas Gunadarma,Merakit model,0.501280
129,65223,Kabupaten Bekasi,Content writer,Universitas Sumatera Utara (USU),Programming,0.403604
343,85416,Kota Blitar,Content writer,Universitas Bina Nusantara (Binus),Menulis,0.403604
416,33595,Kota Kediri,Content writer,Universitas Sebelas Maret (UNS),Olahraga,0.403604


In [77]:
# Similarity antara sampel X dan Y.
cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

[[1.         0.07161149 0.14322297 ... 0.06362848 0.         0.14322297]
 [0.07161149 1.         0.13333333 ... 0.         0.06262243 0.06666667]
 [0.14322297 0.13333333 1.         ... 0.05923489 0.         0.26666667]
 ...
 [0.06362848 0.         0.05923489 ... 1.         0.5007734  0.05923489]
 [0.         0.06262243 0.         ... 0.5007734  1.         0.        ]
 [0.14322297 0.06666667 0.26666667 ... 0.05923489 0.         1.        ]]
